<a href="https://colab.research.google.com/github/good5229/NLP_Practice/blob/main/NLP_Topic_modeling_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [3]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [4]:
print(dictionary[66])
len(dictionary)

faith


64281

In [5]:
import gensim
NUM_TOPICS = 20 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.016*"said" + 0.009*"went" + 0.007*"people" + 0.007*"know"')
(1, '0.026*"space" + 0.010*"nasa" + 0.009*"research" + 0.008*"center"')
(2, '0.010*"drive" + 0.009*"like" + 0.007*"system" + 0.007*"card"')
(3, '0.013*"list" + 0.010*"send" + 0.008*"request" + 0.007*"mail"')
(4, '0.019*"israel" + 0.017*"jews" + 0.016*"turkish" + 0.016*"armenian"')
(5, '0.010*"nrhj" + 0.007*"wwiz" + 0.006*"bxom" + 0.006*"gizw"')
(6, '0.016*"encryption" + 0.013*"chip" + 0.012*"clipper" + 0.011*"keys"')
(7, '0.012*"thanks" + 0.012*"please" + 0.010*"image" + 0.010*"mail"')
(8, '0.016*"window" + 0.010*"server" + 0.007*"information" + 0.007*"anonymous"')
(9, '0.009*"ground" + 0.006*"water" + 0.006*"wire" + 0.006*"power"')
(10, '0.013*"scsi" + 0.012*"gordon" + 0.011*"pitt" + 0.010*"banks"')
(11, '0.013*"game" + 0.012*"team" + 0.011*"year" + 0.009*"games"')
(12, '0.015*"period" + 0.008*"power" + 0.008*"pittsburgh" + 0.008*"play"')
(13, '0.019*"file" + 0.019*"windows" + 0.014*"version" + 0.014*"available"')
(14,

In [6]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 4.1MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=8f9ad7a65f25b07f5b29ee9eb8d814300eb9a635cbaf973495deeb72c371eb62
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [7]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [8]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(4, 0.1829652), (15, 0.50063723), (16, 0.11632687), (17, 0.18716747)]
1 번째 문서의 topic 비율은 [(8, 0.20953806), (9, 0.026436916), (11, 0.07366139), (15, 0.67085147)]
2 번째 문서의 topic 비율은 [(4, 0.24229538), (15, 0.7431885)]
3 번째 문서의 topic 비율은 [(2, 0.1966192), (6, 0.31609237), (10, 0.08391459), (15, 0.3057693), (18, 0.06303584), (19, 0.024120932)]
4 번째 문서의 topic 비율은 [(0, 0.23306672), (11, 0.3574429), (12, 0.101969846), (15, 0.20391753), (18, 0.075825244)]


In [10]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [11]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,15.0,0.5006,"[(4, 0.18292329), (15, 0.50055355), (16, 0.116..."
1,1,15.0,0.6708,"[(8, 0.20951375), (9, 0.026436256), (11, 0.073..."
2,2,15.0,0.7432,"[(4, 0.24229524), (15, 0.7431886)]"
3,3,6.0,0.3161,"[(2, 0.19616078), (6, 0.31607908), (10, 0.0839..."
4,4,11.0,0.3575,"[(0, 0.23307203), (11, 0.35748944), (12, 0.101..."
5,5,15.0,0.5718,"[(6, 0.049506206), (12, 0.168711), (15, 0.5717..."
6,6,14.0,0.7793,"[(0, 0.012813024), (2, 0.18431684), (8, 0.0141..."
7,7,15.0,0.5556,"[(0, 0.28345534), (4, 0.09032376), (15, 0.5555..."
8,8,15.0,0.5785,"[(7, 0.22014551), (14, 0.1783843), (15, 0.5784..."
9,9,2.0,0.6291,"[(1, 0.01841351), (2, 0.62910694), (15, 0.3408..."
